In [ ]:
%pip install roboflow ultralytics

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")  # Replace with your API key
project = rf.workspace("yolox-qcftu").project("pcb-defect-y2wuo")
dataset = project.version(1).download("yolov8")

In [2]:
print(dataset.location)

/Users/aryan/tensorflow-test/PCB-Defect-1


In [ ]:
import yaml
import os

base_dir = dataset.location 

new_config = {
    'path': base_dir,
    'train': os.path.join(base_dir, "train/images"),
    'val':   os.path.join(base_dir, "test/images"),
    'test':  os.path.join(base_dir, "test/images"),
    'nc': 6,
    'names': ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
}

yaml_path = os.path.join(base_dir, "data.yaml")
with open(yaml_path, 'w') as f:
    yaml.dump(new_config, f)

print(f"data.yaml at: {yaml_path}")
print(f"Train path: {new_config['train']}")
print(f"Val path: {new_config['val']}")

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=10,
    imgsz=640,
    batch=8,
    device='mps',
    plots=True,
    verbose=True
)

print(f"Training finished. Best weights: {results.save_dir}/weights/best.pt")

In [ ]:
import cv2
import glob
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO

test_images = glob.glob(f"{dataset.location}/test/images/*.jpg")
best_weight_path = "/Users/aryan/tensorflow-test/runs/detect/train2/weights/best.pt"

print(f"Found {len(test_images)} test images")
print(f"Model weights: {best_weight_path}")

In [ ]:
def inspect_component(image_path, model_path, conf_threshold=0.25):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading {image_path}")
        return

    height, width, _ = img.shape
    image_area = height * width

    model = YOLO(model_path)
    results = model.predict(img, conf=conf_threshold)[0]

    print(f"\n--- Inspection Report for {os.path.basename(image_path)} ---")
    
    defect_count = 0
    
    for box in results.boxes:
        defect_count += 1
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])

        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)
        
        box_area = (x2 - x1) * (y2 - y1)
        ratio = box_area / image_area
        
        if ratio < 0.005: severity = "Low"
        elif ratio < 0.02: severity = "Medium"
        else: severity = "High"

        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        cv2.putText(img, f"{label} ({severity})", (int(x1), int(y1)-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        print(f"Defect: {label} | Confidence: {conf:.2f} | Severity: {severity} | Center: ({center_x}, {center_y})")

    if defect_count == 0:
        print("No defects detected above confidence threshold.")
    else:
        print(f"\nTotal defects found: {defect_count}")

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Inspection: {os.path.basename(image_path)}")
    plt.show()


if test_images:
    inspect_component(test_images[0], best_weight_path)
else:
    print("No test images found!")

In [ ]:
for img_path in test_images[:3]:
    inspect_component(img_path, best_weight_path)